In [9]:
def cyrillic_to_latin(text):
    with open('../data/kbd cyrillic-latin alphabet table.txt', 'r', encoding='utf-8') as alphabet_table:
        for line in alphabet_table:
            key, value = line.split(':')
            text = text.replace(key, value.replace('\n', ''))
    return text

#### Create dataset

In [10]:
import json
import os


def create_dataset(path_to_books, path_to_dataset):
    with open(path_to_dataset, 'w', encoding='utf8') as f:
        for filename in os.listdir(path_to_books):
            if filename.startswith('kbd-ru'):
                with open(os.path.join(path_to_books, filename), 'r', encoding='utf8') as f_in:
                    for line in f_in:
                        line_kbd, line_ru = line.strip().split('😀')
                        json.dump({ "translation": { "kbd": cyrillic_to_latin(line_kbd), "ru": line_ru } }, f, ensure_ascii=False, sort_keys=True)
                        f.write("\n")

create_dataset('../data/books/', '../data/books/kbd_lat-ru_dataset.jsonl')

In [11]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("json", data_files='../data/books/kbd_lat-ru_dataset.jsonl')
dataset = dataset['train']
# dataset = dataset.train_test_split(test_size=0.05, shuffle=True)

# 95% train, 5% test + validation
train_test_valid = dataset.train_test_split(test_size=0.05)
# Split the 5% test + valid in half test, half valid
test_valid = train_test_valid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_test_valid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

dataset

Using custom data configuration default-180651606494e575


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to C:\Users\anzor\.cache\huggingface\datasets\json\default-180651606494e575\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 19377
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 510
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 510
    })
})

In [12]:
dataset.push_to_hub('anzorq/kbd_lat-ru')

Pushing split train to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]